In [ ]:
# Citrate = 9, Zinc = 7

msi = ImzMLParser(msi_neg.config['imzml'])
for idx, (x, y, z) in enumerate(msi.coordinates):
    mzs, intensities = msi.getspectrum(idx)
    
mzs, intensities

/home/maxi/mambaforge/envs/miit/lib/python3.10/site-packages/pyimzml/ontology/ontology.py:92: UserWarning: Accession IMS:1000046 found with incorrect name "pixel size x". Updating name to "pixel size (x)".
  warn(


(array([123.99555835, 132.00936396, 140.0159356 , 145.05948449,
        146.03823096, 167.02385646, 174.03229973, 174.8240924 ,
        175.01757918, 191.01209831, 214.03784784, 215.02936149,
        306.06731042, 346.05085093, 426.01141753, 505.95653301,
        714.46935743, 716.46500878, 738.46156538, 740.47405025,
        742.49130508, 744.51327092, 747.48038409, 762.45296768,
        764.46763235, 766.4925462 , 770.50802679, 786.45120504,
        788.48505612, 792.51522853, 812.48991244, 816.52096848,
        833.51384048, 834.47820525, 835.4878728 , 838.5217661 ,
        844.53000144, 857.4761022 , 861.51961418, 863.52605786]),
 array([ 62.16478   ,  22.960638  ,  13.899621  ,  17.324766  ,
         30.75327   ,   4.340327  ,   7.436959  ,  16.471289  ,
          7.866908  , 339.6788    ,  26.870985  , 161.84056   ,
         79.47508   ,   1.9060451 ,   1.5317156 ,   2.2875671 ,
          0.64355135,   5.3374605 ,   0.80910045,   0.89085174,
          3.8004286 ,   4.348967  ,   

In [ ]:
msi_output_path = 'msi_neg_reduced'
if not exists(msi_output_path):
    os.mkdir(msi_output_path)
msi = ImzMLParser(msi_neg.config['imzml'])
specs_in = []
intensities_in = []
for idx, (x, y, z) in enumerate(msi.coordinates):
    specs_, intensities_ = msi.getspectrum(idx)
    specs_in.append(specs_[[7, 9]])
    intensities_in.append(intensities_[[7, 9]])

# specs = np.array(specs_in)
# intensities = np.array(intensities_in)
    
output_path = join(msi_output_path, 'reduced_msi.imzML')
with ImzMLWriter(output_path, mz_dtype=np.float64, intensity_dtype=np.float64) as writer:
    for i, coords in enumerate(msi.coordinates):
        intensities = np.array(intensities_in[i]).astype(np.float64)
        mzs = np.array(specs_in[i]).astype(np.float64)
        # if intensities.dtype != np.float64:
        #     intensities = intensities.astype(np.float64)
        writer.addSpectrum(mzs, intensities, coords)
# Now we add additional parameters that imzml skipped.
scan_settings_params = [
    ("max dimension x", "IMS:1000044", msi.imzmldict['max dimension x']),
    ("max dimension y", "IMS:1000045", msi.imzmldict['max dimension y']),
    ("pixel size x", "IMS:1000046", msi.imzmldict['pixel size x']),
    ("pixel size y", "IMS:1000047", msi.imzmldict['pixel size y']),
]
sl = "{http://psi.hupo.org/ms/mzml}"
elem_iterator = etree.parse(output_path)
root = elem_iterator.getroot()
scan_settings_list_elem = root.find('%sscanSettingsList' % sl)
first_scan_setting = scan_settings_list_elem.find('./%sscanSettings' %sl)
first_cv_param_elem = first_scan_setting.findall('./')[0]
for (name, accession, value) in scan_settings_params:
    template_cv_param_elem = deepcopy(first_cv_param_elem)
    template_cv_param_elem.attrib['accession'] = accession
    template_cv_param_elem.attrib['value'] = str(value)
    template_cv_param_elem.attrib['name'] = name
    first_scan_setting.append(template_cv_param_elem)
xml_as_str = etree.tostring(root, pretty_print=True)
with open(output_path, 'wb') as f:
    f.write(xml_as_str)

In [ ]:
from pyimzml.ImzMLParser import ImzMLParser
from pyimzml.ImzMLWriter import ImzMLWriter
from lxml import etree
from copy import deepcopy

msi_output_path = 'msi_pos_reduced'
if not exists(msi_output_path):
    os.mkdir(msi_output_path)
msi = ImzMLParser(msi_pos.config['imzml'])
specs_in = []
intensities_in = []
for idx, (x, y, z) in enumerate(msi.coordinates):
    specs_, intensities_ = msi.getspectrum(idx)
    specs_in.append(specs_[6])
    intensities_in.append(intensities_[6])

specs = np.array(specs_in)
intensities = np.array(intensities_in)
    
output_path = join(msi_output_path, 'reduced_msi.imzML')
with ImzMLWriter(output_path, mz_dtype=np.float64, intensity_dtype=np.float64) as writer:
    for i, coords in enumerate(msi.coordinates):
        intensities = np.array([intensities_in[i]]).astype(np.float64)
        mzs = np.array([specs_in[i]]).astype(np.float64)
        # if intensities.dtype != np.float64:
        #     intensities = intensities.astype(np.float64)
        writer.addSpectrum(mzs, intensities, coords)
# Now we add additional parameters that imzml skipped.
scan_settings_params = [
    ("max dimension x", "IMS:1000044", msi.imzmldict['max dimension x']),
    ("max dimension y", "IMS:1000045", msi.imzmldict['max dimension y']),
    ("pixel size x", "IMS:1000046", msi.imzmldict['pixel size x']),
    ("pixel size y", "IMS:1000047", msi.imzmldict['pixel size y']),
]
sl = "{http://psi.hupo.org/ms/mzml}"
elem_iterator = etree.parse(output_path)
root = elem_iterator.getroot()
scan_settings_list_elem = root.find('%sscanSettingsList' % sl)
first_scan_setting = scan_settings_list_elem.find('./%sscanSettings' %sl)
first_cv_param_elem = first_scan_setting.findall('./')[0]
for (name, accession, value) in scan_settings_params:
    template_cv_param_elem = deepcopy(first_cv_param_elem)
    template_cv_param_elem.attrib['accession'] = accession
    template_cv_param_elem.attrib['value'] = str(value)
    template_cv_param_elem.attrib['name'] = name
    first_scan_setting.append(template_cv_param_elem)
xml_as_str = etree.tostring(root, pretty_print=True)
with open(output_path, 'wb') as f:
    f.write(xml_as_str)

/home/maxi/mambaforge/envs/miit/lib/python3.10/site-packages/pyimzml/ontology/ontology.py:92: UserWarning: Accession IMS:1000046 found with incorrect name "pixel size x". Updating name to "pixel size (x)".
  warn(


In [ ]:
msi2 = ImzMLParser(output_path)
for idx, (x, y, z) in enumerate(msi2.coordinates):
    mzs, intensities = msi2.getspectrum(idx)
    break
intensities

/home/maxi/mambaforge/envs/miit/lib/python3.10/site-packages/pyimzml/ontology/ontology.py:92: UserWarning: Accession IMS:1000046 found with incorrect name "pixel size x". Updating name to "pixel size (x)".
  warn(


array([7.18880892])

In [ ]:
# msi = ImzMLParser(output_path)
msi = ImzMLParser(msi_pos.config['imzml'])
for idx, (x, y, z) in enumerate(msi.coordinates):
    mzs, intensities = msi.getspectrum(idx)
    break
intensities

array([20.419807  , 18.614344  ,  1.3428799 ,  0.32686317,  5.5206733 ,
        8.54752   ,  7.188809  ,  5.4473343 ,  0.49293998,  0.21787834,
        1.215228  ,  1.5868512 ,  1.5627059 ,  0.7452728 , 10.433499  ,
        4.371694  ,  4.716074  ,  2.5100682 ,  6.657046  ,  6.5919347 ,
        3.534585  ,  2.2767746 ,  6.782971  ,  3.2988572 ,  2.102648  ,
        1.8527583 ,  4.61825   ,  1.2757188 ,  2.4360347 ,  1.7121892 ,
        1.4887929 ], dtype=float32)

In [ ]:
# Only save spermine at pos 6.

# with open('../data/msi_neg/neg_ion_labels.txt') as f:
#     msi_pos_labels = [x.strip() for x in f.readlines()]
# msi_pos_labels
identified_peaks_df = pd.read_csv('../data/msi_neg/IdentifiedPeaks.csv')
identified_peaks_df

,m/z;Interval Width (+/- Da);Color;Name
0,124.02;0.049608000000006;#ff0000;Taurine
1,132.03;0.052811999999989;#33a02c;Aspartate
2,140.02;0.056007999999991;#6a33c2;PEA
3,145.07;0.058028000000007;#ff7f00;Glutamine
4,146.06;0.058424000000002;#ffd700;Glutamate
5,167.03;0.066811999999999;#a6cee3;Urate
6,174.04;0.069615999999996;#fb6496;NAA
7,174.83;0.069931999999994;#b2df8a;Zinc
8,175.02;0.070008000000001;#cab2d6;Ascorbate
9,191.02;0.076407999999986;#fdbf6f;Citrate


In [ ]:
import pandas as pd

df = pd.read_csv('~/Documents/miit_test_data/gsea_data_SpTr_Morten_jun2023.txt', sep='\t')
idxs = df['index'].to_list()
idxs = [x for x in idxs if x.startswith('V10A20_008_C')]
df = df[df['index'].isin(idxs)]
df.set_index('index', inplace=True)
df.rename(index={x: x.rsplit('_', maxsplit=1)[-1] for x in df.index}, inplace=True)
csgs_df = df['Citsper']
csgs_df.head()

index
AAACGAGACGGTTGAT-1     92.449139
AAACTGCTGGCTCCAA-1    367.690330
AAAGGGATGTAGCAAG-1    449.198135
AAAGTGTGATTTATCT-1     18.226352
AAAGTTGACTCCCGTA-1    147.569951
Name: Citsper, dtype: float64

In [ ]:
df2 = st.table.data.copy()
df2['CSGS'] = csgs_df
df2.fillna(0, inplace=True)
df2

,x,y,int_idx,CSGS
0,,,,
ACGCCTGACACGCGCT-1,192.613751,192.613751,1,0.0
TACCGATCCAACACTT-1,213.425830,204.856150,2,0.0
ATTAAAGCGGACGAGC-1,192.613751,216.690469,3,0.0
GATAAGGGACGATTAG-1,213.425830,228.524789,4,0.0
GTGCAAATCACCAATA-1,192.613751,240.359108,5,0.0
...,...,...,...,...
GAACGTTTGTATCCAC-1,1778.820633,1652.315839,4988,0.0
ATTGAATTCCCTGTAG-1,1758.416634,1664.150158,4989,0.0
TACCTCACCAATTGTA-1,1778.820633,1675.984478,4990,0.0


In [ ]:
indexes = df['index'].to_list()
indexes = [x.rsplit('_', maxsplit=1)[0] for x in indexes]
indexes = list(set(indexes))
indexes

['V19S23_106_C',
 'V19S23_002_C',
 'V10A20_011_B',
 'V19S23_108_B',
 'V10A20_008_D',
 'V10A20_007_D',
 'V10A20_008_A',
 'V10A20_009_C',
 'V10A20_007_C',
 'V19S23_106_A',
 'V10A20_007_B',
 'V10A20_011_D',
 'V19S23_106_B',
 'V19S23_002_D',
 'V10A20_009_A',
 'V19S23_107_A',
 'V10A20_009_B',
 'V19S23_002_A',
 'V19S23_108_A',
 'V10A20_011_C',
 'V10A20_007_A',
 'V10A20_011_A',
 'V10A20_009_D',
 'V10A20_008_C',
 'V19S23_107_D',
 'V10A20_008_B',
 'V19S23_108_C',
 'V19S23_108_D',
 'V19S23_107_C',
 'V19S23_106_D',
 'V19S23_107_B',
 'V19S23_002_B']

In [ ]:
import numpy as np

def sample_seq(length=16):
    alphabet = ['A', 'C', 'G', 'T']
    seq_prefix = ''.join([alphabet[np.random.randint(4)] for _ in range(length)])
    return f'{seq_prefix}-1'

In [ ]:
new_seqs = [sample_seq() for _ in range(df2.shape[0])]
df2['new_index'] = new_seqs
df2

,x,y,int_idx,CSGS,new_index
0,,,,,
ACGCCTGACACGCGCT-1,192.613751,192.613751,1,0.0,CGTGCCCAGTCCCTTT-1
TACCGATCCAACACTT-1,213.425830,204.856150,2,0.0,GACTCGTAGGTATTGA-1
ATTAAAGCGGACGAGC-1,192.613751,216.690469,3,0.0,CGAAAAACTCCGTAGC-1
GATAAGGGACGATTAG-1,213.425830,228.524789,4,0.0,GATTTAGGCGGGAATA-1
GTGCAAATCACCAATA-1,192.613751,240.359108,5,0.0,CTTCAGCCAGTAACGA-1
...,...,...,...,...,...
GAACGTTTGTATCCAC-1,1778.820633,1652.315839,4988,0.0,GAACACTATAGATATT-1
ATTGAATTCCCTGTAG-1,1758.416634,1664.150158,4989,0.0,TTTGGTTTCCTCCCTA-1
TACCTCACCAATTGTA-1,1778.820633,1675.984478,4990,0.0,CCCTTTGATCGCGTTC-1


In [ ]:
np.random.randint(4)

3

In [ ]:
os.listdir(join(section_directory, '2', 'spatial_transcriptomics', 'spatial', 'tissue_positions_list.csv'))

['detected_tissue_image.jpg',
 'aligned_fiducials.jpg',
 'tissue_lowres_image.png',
 'scalefactors_json.json',
 'tissue_positions_list.csv',
 'tissue_hires_image.png']

In [ ]:
# df2.set_index('new_index')[['x', 'y']].to_csv(join('~/Documents/miit_test_data/test_data_publication_backup', '2', 'spatial_transcriptomics', 'spatial', 'tissue_positions_list.csv'))

In [ ]:
df_tp = pd.read_csv(join('~/Documents/miit_test_data/test_data_publication_backup', '2', 'spatial_transcriptomics', 'spatial', 'tissue_positions_list.csv'), header=None)
# df_tp['new_index'] = new_seqs
old_seqs = df_tp[0].to_list()
df_tp[0] = new_seqs
mapping_old_2_new = {x: y for (x, y) in zip(old_seqs, new_seqs)}
df_tp.to_csv(join(section_directory, '2', 'spatial_transcriptomics', 'spatial', 'tissue_positions_list.csv'), header=None, index=None)

In [ ]:
df_gsgs = df2.rename(index=mapping_old_2_new)['CSGS']
df_gsgs

0
CGTGCCCAGTCCCTTT-1    0.0
GACTCGTAGGTATTGA-1    0.0
CGAAAAACTCCGTAGC-1    0.0
GATTTAGGCGGGAATA-1    0.0
CTTCAGCCAGTAACGA-1    0.0
                     ... 
GAACACTATAGATATT-1    0.0
TTTGGTTTCCTCCCTA-1    0.0
CCCTTTGATCGCGTTC-1    0.0
TTCGTAGGATCAACTG-1    0.0
TGACTCGAATTAAGGA-1    0.0
Name: CSGS, Length: 4992, dtype: float64

In [ ]:
df_gsgs.to_csv(join(section_directory, '2', 'spatial_transcriptomics', 'csgs.csv'))

In [ ]:
[len(x) for x in df2.index.to_list()]

[18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,


In [ ]:
st_idxes = st.table.data.index.to_list()

In [ ]:
for indexes_ in indexes:
    long_indexes = df['index'].to_list()
    long_indexes = [x for x in long_indexes if x.startswith(indexes_)]
    sub_df = df[df['index'].isin(long_indexes)]['index'].to_list()
    long_st_idxes = [f'{indexes_}_{x}' for x in st_idxes]
    counter = 0
    for i in sub_df:
        if i in long_st_idxes:
            counter += 1
    print(indexes_, counter)

V19S23_106_C 661
V19S23_002_C 450
V10A20_011_B 606
V19S23_108_B 592
V10A20_008_D 711
V10A20_007_D 642
V10A20_008_A 659
V10A20_009_C 647
V10A20_007_C 627
V19S23_106_A 538
V10A20_007_B 734
V10A20_011_D 647
V19S23_106_B 599
V19S23_002_D 561
V10A20_009_A 711
V19S23_107_A 558
V10A20_009_B 697
V19S23_002_A 673
V19S23_108_A 551
V10A20_011_C 692
V10A20_007_A 587
V10A20_011_A 611
V10A20_009_D 751
V10A20_008_C 737
V19S23_107_D 537
V10A20_008_B 681
V19S23_108_C 566
V19S23_108_D 556
V19S23_107_C 528
V19S23_106_D 560
V19S23_107_B 594
V19S23_002_B 590


In [ ]:
# warped_sections = []

# options = RegistrationOptions()
# options.affine_registration_options.n_threads = 8
# options.nonrigid_registration_options.n_threads = 8
    
# registerer = GreedyFHist(path_to_greedy='mwess89/greedy:1.3.0-alpha', use_docker_container=True)

# for i in range(len(section_list)-1):
#     print(i)
#     moving_section = section_list[i]
#     fixed_section = section_list[i+1]
#     moving_image = moving_section.reference_image.data
#     fixed_image = fixed_section.reference_image.data

#     moving_img_mask = moving_section.get_annotations_by_names('tissue_mask').data
#     fixed_img_mask = fixed_section.get_annotations_by_names('tissue_mask').data

#     moving_lms = moving_section.get_annotations_by_names('landmarks').data
#     fixed_lms = fixed_section.get_annotations_by_names('landmarks').data



#     registration_result = registerer.register(
#         moving_img=moving_image,
#         fixed_img=fixed_image,
#         options=options
#     )

#     warped_image = registerer.transform_image(moving_image, registration_result.registration.forward_transform)
#     warped_lms = registerer.transform_pointset(moving_lms[['x', 'y']].to_numpy(), registration_result.registration.backward_transform)
#     warped_lms = pd.DataFrame(warped_lms, columns=['x', 'y'])
#     warped_lms['label'] = moving_lms['label'].copy()
#     warped_sections.append(((moving_image, moving_lms), (fixed_image, fixed_lms), (warped_image, warped_lms)))